In [3]:
#libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/swaru/Desktop/EMG My work/AugmentedAlzheimerDataset",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=[176, 208],
    batch_size=16,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/swaru/Desktop/EMG My work/AugmentedAlzheimerDataset",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=[176, 208],
    batch_size=16 ,
)

Found 33984 files belonging to 4 classes.
Using 27188 files for training.
Found 33984 files belonging to 4 classes.
Using 6796 files for validation.


In [5]:
class_names = ['MildDementia', 'ModerateDementia', 'NonDementia', 'VeryMildDementia']
train_ds.class_names = class_names
val_ds.class_names = class_names

NUM_CLASSES = len(class_names)
NUM_CLASSES

4

In [6]:
def one_hot_label(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(one_hot_label, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(one_hot_label, num_parallel_calls=AUTOTUNE)

In [7]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
train_ds = train_ds.take(5000)
val_ds = val_ds.take(1000)

In [9]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    
    return block

In [10]:
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [11]:
IMAGE_SIZE = [176, 208]
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(*IMAGE_SIZE, 3)),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [12]:
model = build_model()
METRICS = [tf.keras.metrics.AUC(name='auc'),tf.keras.metrics.CategoricalAccuracy(name='acc')]
model.compile(optimizer='adam',loss = tf.losses.CategoricalCrossentropy(),metrics=METRICS)

In [ ]:
fitmodel = model.fit(train_ds, validation_data=val_ds,epochs=6)

Epoch 1/6
1700/1700 [==============================] - 9560s 6s/step - loss: 1.1684 - auc: 0.7539 - acc: 0.4764 - val_loss: 1.0996 - val_auc: 0.8239 - val_acc: 0.5218
Epoch 2/6
1700/1700 [==============================] - 1356s 797ms/step - loss: 0.8409 - auc: 0.8698 - acc: 0.6170 - val_loss: 0.7900 - val_auc: 0.8786 - val_acc: 0.6059
Epoch 3/6
1700/1700 [==============================] - 1374s 808ms/step - loss: 0.7469 - auc: 0.8979 - acc: 0.6688 - val_loss: 0.6378 - val_auc: 0.9255 - val_acc: 0.7087
Epoch 4/6
 170/1700 [==>...........................] - ETA: 16:25 - loss: 0.6819 - auc: 0.9160 - acc: 0.6985